In [1]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences


In [2]:
def boyer_moore_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0    
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            num_character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
        num_alignments += 1
    return occurrences, num_alignments, num_character_comparisons

In [3]:
from bm_preproc import BoyerMoore

In [4]:
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)
#([40], 12, 15)

[40] 12 15


In [5]:
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)
#([0, 19], 5, 18)

[0, 19] 5 18


In [6]:
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

In [7]:
def naive_with_counts(p, t):
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            num_character_comparisons += 1
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        num_alignments += 1
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences, num_alignments, num_character_comparisons



In [8]:
p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)
#([40], 41, 46)

[40] 41 46


In [9]:
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)
#([0, 19], 20, 35)

[0, 19] 20 35


In [10]:
import bisect
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [11]:
def queryIndex(p, t, index):
    k = index.k
    offsets = list()
    for i in index.query(p):
        if p[k:] == t[i+k: i + len(p)]:
            offsets.append(i)
    return offsets

In [12]:
t = "GCTACGATCTAGAATCTA"
p = "TCTA"

In [13]:
index = Index(t, 2)
print(queryIndex(p, t, index))

[7, 14]


In [14]:
t[7:11]

'TCTA'

In [15]:
def naive_2mm(p, t, total_mismatch=2):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        mismatch_count = 0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                mismatch_count += 1
                if mismatch_count > total_mismatch:
                    match = False
                    break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

In [16]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [17]:
human_1 = readGenome("chr1.GRCh38.excerpt.fasta")

In [18]:
p = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, human_1)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 799954 984143


In [21]:
p = "GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG"
lowercase_alphabet = 'ACGT'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, human_1)
print(occurrences, num_alignments, num_character_comparisons)

[56922] 127974 165191


In [22]:
def approximate_match_with_bm(p, t, n):
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()
    num_hits = 0
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        p_bm = BoyerMoore(p[start:end], alphabet='ACGT')
        matches = boyer_moore(p[start:end], p_bm, t)
        num_hits += len(matches)
        # Extend matching segments to see if whole p matches
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches), num_hits

In [60]:
p = "GGCGCGGTGGCTCACGCCTGTAAT"
all_matches_bm, num_hits_bm = approximate_match_with_bm(p, human_1, 2)
all_matches_bm.sort()
print(len(all_matches_bm), all_matches_bm)

19 [56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]


In [61]:
print(all_matches_bm)

[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]


In [62]:
def approximate_match_with_index(p, t, index, n):
    segment_length = int(round(len(p) / (n+1)))
    all_matches = set()
    num_hits = 0
    for i in range(n+1):
        start = i*segment_length
        end = min((i+1)*segment_length, len(p))
        matches = index.query(p[start:end])
        num_hits += len(matches)
        # Extend matching segments to see if whole p matches
        for m in matches:
            if m < start or m-start+len(p) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[m-start+j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            if mismatches <= n:
                all_matches.add(m - start)
    return list(all_matches), num_hits

In [63]:
p = "GGCGCGGTGGCTCACGCCTGTAAT"
index = Index(human_1, 8)
all_matches_in, num_hits_in = approximate_match_with_index(p, human_1, index, 2)
all_matches_in.sort()

In [64]:
print(len(all_matches_in), num_hits_in)

19 90


In [65]:
print(all_matches_bm)
print(all_matches_in)

[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]
[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]


In [66]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, human_1)
print(occurrences, num_alignments, num_character_comparisons)

[56922, 262042, 364263, 657496, 717706] 799977 984116


In [67]:
p = "GGCGCGGTGGCTCACGCCTGTAAT"
index = Index(human_1, 8)
all_matches_in, num_hits_in = approximate_match_with_index(p, human_1, index, 2)
print(len(all_matches_in), num_hits_in)

19 90


In [68]:
p = "GGCGCGGTGGCTCACGCCTGTAAT"
p_bm = BoyerMoore(p, alphabet='ACGT')
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, human_1)
print(occurrences, num_alignments, num_character_comparisons)

[56922, 262042, 364263, 657496, 717706] 126203 196873


In [70]:
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


In [71]:
ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)


[('AAA', 0), ('TTT', 1)]


In [72]:
p = 'TTATAT'
print(ind.query(p[0:]))


[]


In [74]:
print(ind.query(p[1:]))

[1]


In [97]:
def query_subseq(p, t, index):
    l1 = subseq_ind.query(p[0:])
    l2 = subseq_ind.query(p[1:])
    l3 = subseq_ind.query(p[2:])        
    print(l1)
    print(l2)
    print(l3)
    return l1, len(l1) + len(l2) + len(l3)

In [98]:
t = 'to-morrow and to-morrow and to-morrow creeps in this petty pace'
p = 'to-morrow and to-morrow '
subseq_ind = SubseqIndex(t, 8, 3)
occurrences, num_index_hits = query_subseq(p, t, subseq_ind)
print(occurrences)
print(num_index_hits)
#[0, 14] and 6

[0, 14]
[1, 15]
[2, 16]
[0, 14]
6


In [99]:
t[0: len(p)]

'to-morrow and to-morrow '

In [100]:
t[1: 1+len(p)]

'o-morrow and to-morrow a'

In [ ]:
t = 'to-morrow and to-morrow and to-morrow creeps in this petty pace'
p = 'to-morrow and to-morrow '
subseq_ind = SubseqIndex(t, 8, 3)
occurrences, num_index_hits = query_subseq(p, t, subseq_in

In [88]:
subseq_ind = SubseqIndex(t, 8, 3)
print(subseq_ind.index)

[(' -rwntmr', 13), (' -rwrpit', 27), (' doooa -', 9), (' doooce ', 23), (' esnh t ', 37), ('-rwntmr ', 2), ('-rwntmr ', 16), ('-rwrpits', 30), ('a -rwntm', 10), ('a -rwrpi', 24), ('ce  iptp', 38), ('doooa -r', 12), ('doooce  ', 26), ('e  iptpe', 41), ('esnh t c', 40), ('mr doooa', 3), ('mr doooc', 17), ('mr esnh ', 31), ('ntmr doo', 11), ('ntmr esn', 25), ('oa -rwnt', 7), ('oa -rwrp', 21), ('oce  ipt', 35), ('ooa -rwn', 4), ('ooa -rwr', 18), ('ooce  ip', 32), ('oooa -rw', 1), ('oooa -rw', 15), ('oooce  i', 29), ('r doooa ', 6), ('r doooce', 20), ('r esnh t', 34), ('rpitseya', 39), ('rwntmr d', 5), ('rwntmr e', 19), ('rwrpitse', 33), ('tmr dooo', 0), ('tmr dooo', 14), ('tmr esnh', 28), ('wntmr do', 8), ('wntmr es', 22), ('wrpitsey', 36)]


In [90]:
print(subseq_ind.query(p[1:]))

[1, 15]


In [91]:
t[1:1+len(p)]

'o-morrow and to-morrow a'

In [92]:
p[1:]

'o-morrow and to-morrow '

In [104]:
t = open('1110.txt.utf-8').read()
p = 'English measure backward'
subseq_ind = SubseqIndex(t, 8, 3)
occurrences, num_index_hits = query_subseq(p, t, subseq_ind)
print(occurrences)
#[135249]

[132574]
[132575]
[132576]
[132574]


In [103]:
print(num_index_hits)


3


In [106]:
t[135249: 135249 + len(p)]

'recover.\n  BASTARD. Who '

In [107]:
t[132574: 132574 + len(p)]

'English measure backward'

In [108]:
t = human_1
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
subseq_ind = SubseqIndex(t, 8, 3)
occurrences, num_index_hits = query_subseq(p, t, subseq_ind)
print(num_index_hits)


[56922, 67486, 83863, 84641, 84775, 124024, 147558, 191452, 199607, 262042, 262174, 273669, 322735, 364263, 421354, 454332, 465647, 471966, 472634, 489019, 558456, 579737, 596898, 635931, 651523, 657496, 658702, 681737, 707151, 712449, 717706, 719418, 719557, 746620, 747359]
[22398, 32640, 56923, 84642, 100012, 108111, 137575, 147559, 151719, 160163, 160730, 262043, 273670, 364264, 366819, 421222, 429300, 465648, 479031, 551135, 551828, 635932, 657497, 681738, 717707, 724928, 745641, 783347, 794643]
[23005, 56924, 141048, 160731, 191454, 262044, 349191, 364265, 429301, 657498, 704733, 717708, 724929, 747361, 766421]
79
